### 필요한 데이터 불러오기

In [1]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### 생성할 문장 조건 만들어주기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


### 전처리 해주는 과정

In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

### 토큰화 시켜주기

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, ㅠ
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    # maxlen=15로 설정
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa33c9d6b50>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


### test데이터 와 train 데이터 나누기

In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (140599, 14)
Target Train:  (140599, 14)


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 128
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

### 학습시키기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(128, 14, 12001), dtype=float32, numpy=
array([[[ 1.72135478e-04,  1.43189987e-04,  1.17503070e-04, ...,
          9.23977277e-05, -1.45006445e-04,  1.62355747e-04],
        [ 5.87773335e-04,  5.05313568e-04,  1.19454016e-04, ...,
          2.51447695e-04, -3.23475506e-05,  3.29384638e-04],
        [ 7.44739489e-04,  4.03371669e-04, -1.49949992e-04, ...,
          4.95014945e-04, -1.50598062e-04,  3.28486756e-04],
        ...,
        [-4.50142310e-04, -1.07869855e-03, -1.79403403e-03, ...,
          2.33324803e-03,  2.82177672e-04,  9.52906033e-04],
        [-2.66919786e-04, -1.45891914e-03, -2.14467151e-03, ...,
          2.54980964e-03,  5.41327056e-04,  6.09732058e-04],
        [-3.26174595e-05, -1.79685932e-03, -2.43530353e-03, ...,
          2.72314972e-03,  7.74132670e-04,  2.32239923e-04]],

       [[ 1.72135478e-04,  1.43189987e-04,  1.17503070e-04, ...,
          9.23977277e-05, -1.45006445e-04,  1.62355747e-04],
        [ 2.26452059e-04,  1.86512756e-04,  2

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [13]:
#optimizer = tf.keras.optimizers.Adam()
#Loss
#loss = tf.keras.losses.SparseCategoricalCrossentropy(
    #from_logits=True, reduction='none')
#model.compile(loss=loss, optimizer=optimizer)
#model.fit(enc_train, dec_train =(enc_val),epochs=10, batch_size=512)

### 로스값 확인하기

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),epochs=10, batch_size=128)

Epoch 1/10
1099/1099 [==============================] - 298s 269ms/step - loss: 3.4593 - val_loss: 3.0768
Epoch 2/10
1099/1099 [==============================] - 296s 270ms/step - loss: 2.9067 - val_loss: 2.8341
Epoch 3/10
1099/1099 [==============================] - 296s 269ms/step - loss: 2.5939 - val_loss: 2.6565
Epoch 4/10
1099/1099 [==============================] - 296s 270ms/step - loss: 2.2841 - val_loss: 2.5284
Epoch 5/10
1099/1099 [==============================] - 297s 270ms/step - loss: 1.9880 - val_loss: 2.4403
Epoch 6/10
1099/1099 [==============================] - 297s 270ms/step - loss: 1.7184 - val_loss: 2.3833
Epoch 7/10
1099/1099 [==============================] - 297s 270ms/step - loss: 1.4892 - val_loss: 2.3581
Epoch 8/10
1099/1099 [==============================] - 297s 270ms/step - loss: 1.3029 - val_loss: 2.3637
Epoch 9/10
1099/1099 [==============================] - 298s 271ms/step - loss: 1.1616 - val_loss: 2.3864
Epoch 10/10
1099/1099 [=======================

### 잘만들어졌는지 테스트하기

In [29]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [30]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

## 회고

#### 이번 프로잭트를 진행하는데 loss값을 처음 구했을때 2.7이 나와서 처음에는 에포크값을 늘려서 학습시키는 방법을 생각해 봤지만 이 방법은 이번 프로잭트를 진행할때는 사용하지 못하는 방법이라 제외 시켰습니다.                                                                                                                   두번째로는 hidden size, batch size를 바꿔보는 방법을 생각하고 적용해봤는데 처음에는 효과가 있는줄 알았는데 마지막으로 갈수록 loss값이 올라갔습니다.                                                                                                                                                                                                                 아마 계속해서 loss값을 구하려고 에포크를 돌리다 보니 오버피팅이 돼서 그런게 아닐까 라는 생각을 했고 이 문제를 해결해보려고 했지만 도저히 방법을 못 찾아서 이대로 제출 하게 됐습니다. 이 프로잭트를 제출한뒤에도 오버피팅 해결 방법을 끝까지 생각해보겠습니다.
